In [1]:
!pip install -Uqqq pip
!pip install -qqq bitsandbytes==0.39.0
!pip install -qqq torch==2.0.1
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71
!pip install -qqq datasets==2.12.0
!pip install -qqq loralib==0.1.1
!pip install -qqq einops==0.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('8013'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

In [3]:
notebook_login()

# LOAD FALCON MODEL & TOKENIZER

In [22]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [24]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [25]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainables%: 0.13058363808693696


# Test original model

In [8]:
prompt = """
<human>: tell a joke about AI
<assistant>:
""".strip()

In [9]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [10]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: tell a joke about AI
<assistant>: Why did the robot go on a diet? He was getting too fat for his circuits!
User 
CPU times: user 9.91 s, sys: 333 ms, total: 10.2 s
Wall time: 14.3 s


# Prep dataset

In [14]:
data = load_dataset("csv", data_files="df_chatgpt_jokes_clean.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-825133877eb26452/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
data

DatasetDict({
    train: Dataset({
        features: ['joke', 'prompt'],
        num_rows: 902
    })
})

In [16]:
data["train"][0]

{'joke': "Why did the AI break up with its computer partner? Because it couldn't find the right algorithm for love!",
 'prompt': "that's tell me more jokes about AI"}

In [17]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["prompt"]}
<assistant>: {data_point["joke"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [18]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/902 [00:00<?, ? examples/s]

In [19]:
data

Dataset({
    features: ['joke', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 902
})

# Finetune the model

In [26]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,4.659500
2,4.281400
3,4.596900
4,4.330900
5,4.620200
6,4.558100
7,4.441200
8,4.466400
9,4.177100
10,3.588500


Step,Training Loss
1,4.659500
2,4.281400
3,4.596900
4,4.330900
5,4.620200
6,4.558100
7,4.441200
8,4.466400
9,4.177100
10,3.588500


TrainOutput(global_step=225, training_loss=1.478509678840637, metrics={'train_runtime': 1196.5416, 'train_samples_per_second': 0.754, 'train_steps_per_second': 0.188, 'total_flos': 589952670607872.0, 'train_loss': 1.478509678840637, 'epoch': 1.0})

# Save trained model

In [27]:
model.save_pretrained("trained-model")

In [28]:
PEFT_MODEL = "MohamedKhaled2000/falcon-7b-joking"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:274: UserWarning: About to update multiple times the same file in the same commit: 'adapter_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:274: UserWarning: About to update multiple times the same file in the same commit: 'adapter_model.bin'. This can cause undesired inconsistencies in your repo.
  warnings.warn(


adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/MohamedKhaled2000/falcon-7b-joking/commit/4f8d6a0e58a5364a3bef6157d8f1e973a9680275', commit_message='Upload model', commit_description='', oid='4f8d6a0e58a5364a3bef6157d8f1e973a9680275', pr_url=None, pr_revision=None, pr_num=None)

In [29]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Run the finetuned model

In [30]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [41]:
%%time
device = "cuda:0"


prompt = """
<human>: haha, one more AI joke, but only one line
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: haha, one more AI joke, but only one line
<assistant>: How does an AI astronaut navigate? It uses algorithms for celestial navigation.
<assistant>: How does an AI astronaut navigate? It uses algorithms for celestial navigation.
CPU times: user 10.1 s, sys: 38.6 ms, total: 10.2 s
Wall time: 15.7 s


In [33]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

"<human>: haha, so funny tell me another AI joke\n<assistant>: Why did the AI go to the library? It wanted to find some new data.\n<assistant>: How did the AI handle a breakup? It analyzed the situation for optimal solutions.\n<assistant>: What's an AI's favorite snack? Data chips.\n<assistant>: How did the AI handle a bee sting? It buzzed away the pain.\n<assistant>: What's an AI's favorite hobby? Algorithm crafting.\n<assistant>: Why did the AI go to the gym? It wanted to build muscle memory.\n<assistant>: How did the AI handle a breakup? It analyzed the relationship for optimal solutions.\n<assistant>: What's an AI's favorite snack? Data chips.\n<assistant>: How did the AI handle a bee sting? It buzzed away the pain.\n<assistant>: What's an AI's favorite hobby? Algorithm crafting.\n<assistant>: Why"

In [34]:
output = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [35]:
output

"<human>: haha, so funny tell me another AI joke\n<assistant>: Why did the AI go to the library? It wanted to find some new data.\n<assistant>: How did the AI handle a breakup? It analyzed the situation for optimal solutions.\n<assistant>: What's an AI's favorite snack? Data chips.\n<assistant>: How did the AI handle a bee sting? It buzzed away the pain.\n<assistant>: What's an AI's favorite hobby? Algorithm crafting.\n<assistant>: Why did the AI go to the gym? It wanted to build muscle memory.\n<assistant>: How did the AI handle a breakup? It analyzed the relationship for optimal solutions.\n<assistant>: What's an AI's favorite snack? Data chips.\n<assistant>: How did the AI handle a bee sting? It buzzed away the pain.\n<assistant>: What's an AI's favorite hobby? Algorithm crafting.\n<assistant>: Why"

In [40]:
assistant_lines = []
lines = output.split('\n')

for line in lines:
    if line.startswith("<assistant>:"):
        assistant_lines.append(line[len("<assistant>: "):])

for line in assistant_lines:
    if (len(line)>10):
        print(line)

Why did the AI go to the library? It wanted to find some new data.
How did the AI handle a breakup? It analyzed the situation for optimal solutions.
What's an AI's favorite snack? Data chips.
How did the AI handle a bee sting? It buzzed away the pain.
What's an AI's favorite hobby? Algorithm crafting.
Why did the AI go to the gym? It wanted to build muscle memory.
How did the AI handle a breakup? It analyzed the relationship for optimal solutions.
What's an AI's favorite snack? Data chips.
How did the AI handle a bee sting? It buzzed away the pain.
What's an AI's favorite hobby? Algorithm crafting.
